In [6]:
#create a requirements file

try:
    import dash
except:
    ! pip install dash>=2.7.0
    import dash

from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate

try:
    import pandas as pd
except:
    ! pip install pandas
    import pandas as pd
    
import numpy as np
from datetime import datetime as dt
import base64

try:
    import plotly.express as px
except:
    ! pip install plotly
    import plotly.express as px

try:
    import nbformat
except:
    ! pip install nbformat>=4.2.0
    import nbformat

#from plotly import graph_objs as go
#from plotly.graph_objs import *

from random import randint

ImportError: cannot import name 'Dash' from 'dash.dependencies' (c:\Users\stegr\miniconda3\lib\site-packages\dash\dependencies.py)

In [105]:
#! pip install dash-leaflet
import dash_leaflet as dl
import dash_leaflet.express as dlx
import geopandas as gpd
import json

In [13]:
df = pd.read_csv(
        "uber-raw-data-apr14.csv"
)  # changing the formatting to the new version in the live app
df[["Date", "Time"]] = df["Date/Time"].str.split(" ", expand=True)
df["Date"] = df["Date"].apply(date_format)
df["Time"] = df["Time"].apply(time_format)
df = df.sample(n=1000)

In [98]:
gdf = gpd.GeoDataFrame(df)
test = gdf.__geo_interface__

In [141]:
app = dash.Dash()

app.layout = html.Div([
    dl.Map([
        dl.TileLayer(),
        # From in-memory geojson. All markers at same point forces spiderfy at any zoom level.
        dl.GeoJSON(data=test, cluster=True, id='map-geojson'),
        dl.MeasureControl(position="topleft", primaryLengthUnit="kilometers", primaryAreaUnit="hectares"),
    ], center=(-37.75, 175.4), zoom=1, style={'height': '50vh'}, id='map'),
    html.Div(id='div-output'),
])
@app.callback(
    Output(component_id='div-output', component_property='children', allow_duplicate=True),
    Input(component_id='map-geojson', component_property='clickData'), 
    prevent_initial_call=True  
)
def return_details(clickData):
    print(clickData)
    return f"{clickData['properties']['Lat']}, {clickData['properties']['Lat']}"

@app.callback(Output("div-output", "children", allow_duplicate=True), [Input("map", "measureData")], prevent_initial_call=True)
def log(data):
    print(data)
    return json.dumps(data)

@app.callback(
    Output(component_id='div-output', component_property='children'),
    Input(component_id='map', component_property='clickData'),
    prevent_initial_call=True
)
def lat_lon(clickData):
    print(clickData)

if __name__ == "__main__":
    app.run_server(jupyter_mode="external", debug=True)

Dash app running on http://127.0.0.1:8050/


{'id': '139520', 'type': 'Feature', 'properties': {'Date/Time': '4/20/2014 18:26:00', 'Lat': 40.5491, 'Lon': -74.1467, 'Base': 'B02598', 'Date': '2014-04-20', 'Time': '18:26:00', 'cluster': False, 'id': 142}, 'geometry': {'type': 'Point', 'coordinates': [-74.1467, 40.5491]}, 'bbox': [-74.1467, 40.5491, -74.1467, 40.5491]}
{'latlng': {'lat': 40.554765353152284, 'lng': -74.21985626220705}, 'layerPoint': {'x': 1338, 'y': 434}, 'containerPoint': {'x': 1241, 'y': 339}}
{'latlng': {'lat': 40.5601125938834, 'lng': -74.29298400878908}, 'layerPoint': {'x': -2693, 'y': 419}, 'containerPoint': {'x': 28, 'y': 119}}
{'latlng': {'lat': 40.53291548215736, 'lng': -74.21942710876466}, 'layerPoint': {'x': -1836, 'y': 836}, 'containerPoint': {'x': 885, 'y': 536}}
{'latlng': {'lat': 40.54661324981587, 'lng': -74.20037269592287}, 'layerPoint': {'x': -1614, 'y': 626}, 'containerPoint': {'x': 1107, 'y': 326}}
{'id': '242059', 'type': 'Feature', 'properties': {'Date/Time': '4/7/2014 6:05:00', 'Lat': 40.6314, 

In [142]:
import pyproj
geodesic = pyproj.Geod(ellps='WGS84')
fwd_azimuth,back_azimuth,distance = geodesic.inv(-73.935, 40.6314, -74.21942710876466, 40.53291548215736)
print(fwd_azimuth,back_azimuth,distance)

-114.33260202889574 65.48236711291466 26447.81886216212


In [2]:
def date_format(x):
    return dt.strptime(x, "%m/%d/%Y").date()


def time_format(x):
    return dt.strptime(x, "%H:%M:%S").time()


def parse_content(contents):
    content_type, content_string = contents.split(",")
    decoded = base64.b64decode(content_string)
    fdi_list = decoded.decode("utf-8").split(",")
    return fdi_list


def create_blank_map():
    fig = px.scatter_mapbox(
        pd.DataFrame({"lat": [], "lon": []}),
        lat="lat",
        lon="lon",
        mapbox_style="carto-darkmatter",
        zoom=1,
    )
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=10), paper_bgcolor="#1E1E1E")
    return fig


def create_map(df):
    max_range = (
        max(max(df["Lat"]) - min(df["Lat"]), max(df["Lon"]) - min(df["Lon"])) * 111
    )
    zoom = 13.5 - np.log(max_range)
    fig = px.scatter_mapbox(
        df, lat=df["Lat"], lon=df["Lon"], mapbox_style="carto-darkmatter", zoom=zoom
    )
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=10), paper_bgcolor="#1E1E1E")
    return fig


def create_blank_chart():
    fig = px.bar().add_annotation(x=10, y=10, text="No Data To Display")
    fig.update_layout(
        margin=dict(l=0, r=0, b=0, t=10),
        paper_bgcolor="#1E1E1E",
        plot_bgcolor="#323130",
        showlegend=False,
        font=dict(color="white"),
    )
    return fig


def create_chart(df):
    fig = px.bar(df["Date"].value_counts())
    fig.update_layout(
        margin=dict(l=0, r=0, b=0, t=10),
        paper_bgcolor="#1E1E1E",
        plot_bgcolor="#323130",
        showlegend=False,
        font=dict(color="white"),
    )
    return fig


app = dash.Dash(
    __name__,
    meta_tags=[{"name": "viewport", "content": "width=device-width"}],
)
app.title = "** GEO RENDERER"
server = app.server


# Layout of Dash App
app.layout = html.Div(
    children=[
        html.Div(
            className="row",
            children=[
                # Column for user controls
                html.Div(
                    className="four columns div-user-controls",
                    children=[
                        html.Div(
                            className="div-for-side-by-side",
                            children=[
                                html.Img(
                                    className="logo",
                                    src=app.get_asset_url("dash-logo-new.png"),
                                ),
                                html.H2("## GEO RENDERER"),
                            ],
                        ),
                        html.Br(),
                        dcc.Upload(
                            id="upload-data",
                            children=html.Div(
                                [
                                    "Drag and Drop or ",
                                    html.A("Select FDI File"),
                                    " to begin.",
                                ]
                            ),
                            style={
                                "width": "100%",
                                "height": "60px",
                                "lineHeight": "60px",
                                "borderWidth": "1px",
                                "borderStyle": "dashed",
                                "borderRadius": "7px",
                                "textAlign": "center",
                                "margin": "5px",
                                "backgroundColor": "#43449444",
                                "hover-backgroundColor": "#555555",
                            },
                            # Allow multiple files to be uploaded
                            multiple=False,
                        ),
                        html.Div(
                            className="div-for-text-updates",
                            id="fdi-output",
                            children=["Once loaded, your FDIs will be shown here."],
                        ),
                        html.Div(
                            className="div-for-text-updates",
                            id="df-info",
                            children=[
                                "Select time frame for initial dataframe creation - it will be created from now - hours."
                            ],
                        ),
                        html.Div(
                            className="div-for-side-by-side",
                            children=[
                                dcc.Dropdown(
                                    id="generate-df-delta",
                                    options=[
                                        {
                                            "label": str(n),
                                            "value": str(n),
                                        }
                                        for n in range(1, 25, 1)
                                    ],
                                    multi=False,
                                    maxHeight=120,
                                    placeholder="Time delta for dataframe (hrs)",
                                    style={
                                        "width": "275px",
                                    },
                                ),
                                html.Button(
                                    "Generate Dataframe",
                                    className="button",
                                    id="generate-df-button",
                                    n_clicks=0,
                                ),
                            ],
                        ),
                        html.P(
                            """Auto refresh options - time delta will create a new dataframe using now-hours, per your interval:"""
                        ),
                        html.P(
                            """Note - checking Auto Refresh will trigger an immediate data refresh:"""
                        ),
                        html.Div(
                            className="div-for-side-by-side",
                            children=[
                                dcc.Dropdown(
                                    id="refresh-frequency",
                                    options=[
                                        {"label": i, "value": i}
                                        for i in range(5, 65, 5)
                                    ],
                                    placeholder="Set frequency of refresh (mins)",
                                    style={
                                        "width": "275px",
                                    },
                                ),
                                dcc.Checklist(
                                    options=["Auto Refresh"],
                                    value=[],
                                    id="refresh-check",
                                ),
                            ],
                        ),
                        dcc.Dropdown(
                            id="refresh-df-delta",
                            options=[{"label": i, "value": i} for i in range(1, 25, 1)],
                            placeholder="Time delta for auto refresh (hrs)",
                            style={
                                "width": "275px",
                            },
                        ),
                        html.P(
                            """
                            Click a GEO point to display report details:
                            """
                        ),
                        html.Div(className="div-for-text-display", id="my-output"),
                        html.Div(
                            className="row",
                            children=[
                                html.Div(
                                    className="div-for-dropdown",
                                    children=[
                                        dcc.Dropdown(
                                            id="chart-selector",
                                            options=["activity types taken from DF"],
                                            placeholder="Select graph display preference (default activity type)",
                                        ),
                                    ],
                                ),
                            ],
                        ),
                        html.Br(),
                        dcc.Markdown(
                            className="contact-details",
                            children=[
                                """
                                            For assistance contact ### Data Science, ##### - ds@email.com
                                        """,
                                """
                                            Main POC ###############
                                        """,
                            ],
                        ),
                        dcc.Interval(
                            id="interval-component", interval=5000, n_intervals=0
                        ),
                    ],
                ),
                # Column for app graphs and plots
                html.Div(
                    className="eight columns div-for-charts bg-grey",
                    children=[
                        dcc.Graph(id="map-graph", figure=create_blank_map()),
                        # html.Div(
                        #     className="div-for-text-updates",
                        #     children=[
                        #         "Select any of the bars on the histogram to apply a map filter."
                        #     ],
                        # ),
                        dcc.Graph(id="histogram", figure=create_blank_chart()),
                    ],
                ),
            ],
        ),
    ]
)


# output details of points when clicked
@app.callback(
    Output(component_id="fdi-output", component_property="children"),
    Input(component_id="upload-data", component_property="contents"),
    State("upload-data", "filename"),
    State("upload-data", "last_modified"),
    prevent_initial_call=True,
)
def handle_upload(
    list_of_contents, list_of_names, list_of_dates
):  # needs some error handling
    global fdi_list
    fdi_list = parse_content(list_of_contents)
    fdi_string = ", ".join(fdi_list)
    return f"Your loaded FDIs are: {fdi_string}"


@app.callback(
    Output(component_id="df-info", component_property="children"),
    [Input(component_id="generate-df-button", component_property="n_clicks")],
    prevent_initial_call=True,
)
def df_updates(n_clicks):
    return ["Acquiring data and generating DF - this may take some time."]


@app.callback(
    [
        Output(
            component_id="map-graph", component_property="figure", allow_duplicate=True
        ),
        Output(
            component_id="histogram", component_property="figure", allow_duplicate=True
        ),
        Output(
            component_id="df-info", component_property="children", allow_duplicate=True
        ),
    ],
    [
        # Input(component_id='generate-df-delta', component_property='clickData'),
        Input(component_id="generate-df-button", component_property="n_clicks")
    ],
    prevent_initial_call=True,
)
def create_df(n_clicks):
    # here will be the API in the live version.
    # this will need to handle the time inputs from the drop downs
    # and receive the data from the drag and drop

    # Need an error capture to ensure FDIs are loaded first
    df = pd.read_csv(
        "uber-raw-data-apr14.csv"
    )  # changing the formatting to the new version in the live app
    df[["Date", "Time"]] = df["Date/Time"].str.split(" ", expand=True)
    df["Date"] = df["Date"].apply(date_format)
    df["Time"] = df["Time"].apply(time_format)
    df = df.sample(n=1000)
    print(fdi_list)
    map_fig = create_map(df)
    chart_fig = create_chart(df)
    text_output = ["Complete!"]
    return map_fig, chart_fig, text_output


@app.callback(
    Output(component_id="interval-component", component_property="interval"),
    Input(component_id="refresh-frequency", component_property="value"),
    prevent_initial_call=True,
)
def update_interval(interval_refresh):
    if interval_refresh!=None:
        return interval_refresh * 1000
    else:
        return 5000


@app.callback(
    [
        Output(component_id="my-output", component_property="children"),
        # Output(component_id='interval-component', component_property='interval')
    ],
    [
        Input(component_id="refresh-check", component_property="value"),
        Input(component_id="interval-component", component_property="n_intervals"),
        State(component_id="refresh-frequency", component_property="value"),
        State(component_id="refresh-df-delta", component_property="value"),
    ],
    prevent_initial_call=True,
)
def auto_refresh_process(checkBox, n_intervals, freq, delta):
    if len(checkBox) > 0: # and there is a valid interval set?
        return refresh_df(freq, delta)
    else:
        raise PreventUpdate


def refresh_df(freq, delta):
    return [
        f"{freq},{delta} - random {randint(0,10)}"
    ]  # change this for create df update map etc


############# take the callbacks above this line for live version #####################


@app.callback(
    Output(
        component_id="my-output", component_property="children", allow_duplicate=True
    ),
    Input(component_id="map-graph", component_property="clickData"),
    prevent_initial_call=True,
)
def update_output_div(clickData):
    return f"{clickData['points'][0]['lat']},{clickData['points'][0]['lon']}"
    # return f"{clickData}"


# update map based on time selection
@app.callback(
    Output(component_id="map-graph", component_property="figure", allow_duplicate=True),
    [
        Input(component_id="chart-selector", component_property="value"),
        Input(component_id="histogram", component_property="clickData"),
    ],
    prevent_initial_call=True,
)
def filter_map(time_picked, selected_data):
    if time_picked:
        temp_df = df[df["Date"] == dt.strptime(time_picked, "%Y-%m-%d").date()]
        max_range = (
            max(
                max(temp_df["Lat"]) - min(temp_df["Lat"]),
                max(temp_df["Lon"]) - min(temp_df["Lon"]),
            )
            * 111
        )
        zoom = 13.5 - np.log(max_range)
        # print(temp_df)
        fig = px.scatter_mapbox(
            temp_df,
            lat=temp_df["Lat"],
            lon=temp_df["Lon"],
            mapbox_style="carto-darkmatter",
            zoom=zoom,
        )
        fig.update_layout(margin=dict(l=0, r=0, b=0, t=10), paper_bgcolor="#1E1E1E")
        return fig
    if selected_data:
        # print(selected_data['points'][0]['label'])
        temp_df = df[
            df["Date"]
            == dt.strptime(selected_data["points"][0]["label"], "%Y-%m-%d").date()
        ]
        max_range = (
            max(
                max(temp_df["Lat"]) - min(temp_df["Lat"]),
                max(temp_df["Lon"]) - min(temp_df["Lon"]),
            )
            * 111
        )
        zoom = 13.5 - np.log(max_range)
        # print(temp_df)
        fig = px.scatter_mapbox(
            temp_df,
            lat=temp_df["Lat"],
            lon=temp_df["Lon"],
            mapbox_style="carto-darkmatter",
            zoom=zoom,
        )
        fig.update_layout(margin=dict(l=0, r=0, b=0, t=10), paper_bgcolor="#1E1E1E")
        return fig


if __name__ == "__main__":
    app.run_server(jupyter_mode="external", debug=True)


Dash app running on http://127.0.0.1:8050/
